In [1]:
from agent.dqn.nn import GNN

gnn = GNN(
    entities=["a", "b", "c"],
    relations=["r1", "r2", "r3"],
    # gcn_layer_params={"type": "vanilla_gcn", "embedding_dim": 4, "num_layers": 2},
)

sum(p.numel() for p in gnn.parameters())


/home/tk/.virtualenvs/agent-room-env-v2-gnn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1698

In [8]:
import torch
gnn.entity_embeddings.shape


torch.Size([3, 8])

In [11]:
gnn.entity_embeddings[torch.tensor([1])].shape

torch.Size([1, 8])

In [12]:
from torch_geometric.data import Data, DataLoader